In [1]:
from nltk.classify import NaiveBayesClassifier, accuracy
from nltk.corpus import names
from string import ascii_lowercase

import pandas as pd

In [2]:
!git clone https://github.com/jvalhondo/spanish-names-surnames /tmp/spanish-surnames

fatal: destination path '/tmp/spanish-surnames' already exists and is not an empty directory.


In [3]:
VOCALS = 'aeiou'
def find_vocal(word, n = 0, set = VOCALS):
    counter = n
    for l in word:
        if l in VOCALS:
            counter -= 1
            if counter == 0:
                return l
    return '_'

In [4]:
S_sonante = 'sz'

In [5]:
def atributos(name):
    assert isinstance(name, str)
    if len(p := name.split(' ')) > 1:
        name = p[0]
    name = name.lower()
    attributes = {
        'first' : name[0],
        'last' : name[-1],
        # 'last2': name[-2],
    }
    try:
        attributes |= { 'last2': name[-2] }
    except Exception as e:
        attributes |= { 'last2': '_' }

    try:
        attributes |= { 'last3': name[-3] }
    except Exception as e:
        attributes |= { 'last3': '_' }

    # try:
    #     attributes |= { 'last4': name[-4] }
    # except Exception as e:
    #     attributes |= { 'last4': '_' }
    # try:
    #     attributes |= { 'first2': name[1] }
    # except Exception as e:
    #     attributes |= { 'first2': '_' }

    attributes |= { f'count({l})': name.count(l) for l in ascii_lowercase }
    attributes |= { f'has({l})': l in name for l in ascii_lowercase }
    attributes |= { f'count': len(name)  }
    attributes |= { f'first_vowel': find_vocal(name, 0) }
    attributes |= { f'last_vowel': find_vocal(name[::-1], 0) }
    attributes |= { f'second_last_vowel': find_vocal(name[::-1], 1) }
    attributes |= { f'same?': find_vocal(name) == find_vocal(name[::-1]) }
    attributes |= { f'same_last?': find_vocal(name[::-1],0) == find_vocal(name[::-1],1) }
    # attributes |= { f's?': find_vocal(name, set = 'sz') != '_' }

    return attributes

In [6]:
tagset = [{'word': word, 'genre': 'male'} for word in names.words('male.txt')]
tagset += [{'word': word, 'genre': 'female'} for word in names.words('female.txt')]

In [7]:
tagset = pd.read_csv('/tmp/spanish-surnames/male_names.csv')
tagset[['genre']] = 'male'
tagset = pd.concat([tagset, pd.read_csv('/tmp/spanish-surnames/female_names.csv')])
tagset[['genre']] = tagset[['genre']].fillna('female')
# tagset = tagset.iloc[:-1]
tagset = tagset.convert_dtypes()
tagset = tagset.dropna()

In [8]:
# random.seed(43)
# random.shuffle(tagset)

In [9]:
# df = pd.DataFrame(tagset)
tagset['attr'] = tagset['name'].apply(atributos)
tagset['attr'] = tagset[['mean_age', 'attr', 'frequency']].apply(lambda x: x['attr'] | { 'age' : x['mean_age'], 'freq': x['frequency']  },axis=1)
tagset

,name,frequency,mean_age,genre,attr
0,ANTONIO,715215,54.6,male,"{'first': 'a', 'last': 'o', 'last2': 'i', 'las..."
1,JOSE,641525,60.0,male,"{'first': 'j', 'last': 'e', 'last2': 's', 'las..."
2,MANUEL,618891,54.0,male,"{'first': 'm', 'last': 'l', 'last2': 'e', 'las..."
3,FRANCISCO,530309,56.0,male,"{'first': 'f', 'last': 'o', 'last2': 'c', 'las..."
4,JUAN,367726,54.9,male,"{'first': 'j', 'last': 'n', 'last2': 'a', 'las..."
...,...,...,...,...,...
24751,ZHIHUI,20,23.0,female,"{'first': 'z', 'last': 'i', 'last2': 'u', 'las..."
24752,ZOILA ESTHER,20,49.9,female,"{'first': 'z', 'last': 'a', 'last2': 'l', 'las..."
24753,ZSANETT,20,28.6,female,"{'first': 'z', 'last': 't', 'last2': 't', 'las..."
24754,ZULEJA,20,45.4,female,"{'first': 'z', 'last': 'a', 'last2': 'j', 'las..."


In [10]:
from sklearn.model_selection import train_test_split
# train, test = train_test_split(tagset, train_size=0.9, random_state=43)
train, test = tagset.iloc[500:], tagset.iloc[:500]

In [11]:
classifier = NaiveBayesClassifier.train(train.iloc[:,[-1,-2]].values.tolist())

In [12]:
test

,name,frequency,mean_age,genre,attr
0,ANTONIO,715215,54.6,male,"{'first': 'a', 'last': 'o', 'last2': 'i', 'las..."
1,JOSE,641525,60.0,male,"{'first': 'j', 'last': 'e', 'last2': 's', 'las..."
2,MANUEL,618891,54.0,male,"{'first': 'm', 'last': 'l', 'last2': 'e', 'las..."
3,FRANCISCO,530309,56.0,male,"{'first': 'f', 'last': 'o', 'last2': 'c', 'las..."
4,JUAN,367726,54.9,male,"{'first': 'j', 'last': 'n', 'last2': 'a', 'las..."
...,...,...,...,...,...
495,JOSE EMILIO,3506,48.9,male,"{'first': 'j', 'last': 'e', 'last2': 's', 'las..."
496,AUGUSTO,3497,54.9,male,"{'first': 'a', 'last': 'o', 'last2': 't', 'las..."
497,MAXIMILIANO,3440,51.1,male,"{'first': 'm', 'last': 'o', 'last2': 'n', 'las..."
498,CEFERINO,3434,63.3,male,"{'first': 'c', 'last': 'o', 'last2': 'n', 'las..."


In [13]:
accuracy(classifier ,test.iloc[:, [-1,-2]].values.tolist())

0.89